In [1]:
# Model comparisons

In [2]:
import cv2
from face_alignment import FaceAlignment, LandmarksType
import os
import urllib.request as urlreq

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff

In [3]:
ds = DataLoader300W(dir_path="300W/sample")
chosen_idx = 4
image, ground_truth_landmarks = ds[chosen_idx]

In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
facealignment_model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device=device, flip_input=False))
opencv_model = OpenCVWrapper()

File exists
File exists


In [6]:
Test(metric=NMEMean, threshold=1).run(model=opencv_model, dataset=ds)

Test "NME_mean" succeeded
               Description: Mean of normalised mean Euclidean distances across images
               Metric: 0.0414 (threshold = 1)
               
               Prediction time: 0.61 s.

In [7]:
Test(metric=NMEMean, threshold=1).run(model=facealignment_model, dataset=ds)

Test "NME_mean" succeeded
               Description: Mean of normalised mean Euclidean distances across images
               Metric: 0.0623 (threshold = 1)
               
               Prediction time: 25.46 s.

In [8]:
from loreal_poc.marks.facial_parts import FacialParts

facial_part = FacialParts.left_half

TestDiff(metric=NMEMean, threshold=1).run(
    model=opencv_model, dataset_ref=ds, dataset_other=CroppedDataLoader(ds, part=facial_part), facial_part=facial_part
)

/home/bazire/loreal-poc/loreal_poc/tests/performance.py:47: RuntimeWarning: Mean of empty slice
  mes = np.nanmean(es, axis=1)


Test "NME_mean" succeeded
               Description: Mean of normalised mean Euclidean distances across images
               Metric: 0.6623 (threshold = 1)
               Prediction fail rate: 0.4
               Prediction time: 0.66 s.

In [9]:
from dataclasses import fields

results = {"OpenCV": {"model": opencv_model}, "FaceAlignment": {"model": facealignment_model}}
for model_name, _model in results.items():
    model = _model["model"]
    for __facial_part in fields(FacialParts):
        pr
        _facial_part = __facial_part.default
        test = TestDiff(metric=NMEMean, threshold=1).run(
            model=model,
            dataset_ref=ds,
            dataset_other=CroppedDataLoader(ds, part=_facial_part),
            facial_part=_facial_part,
        )
        results[model_name][_facial_part.name] = test.metric

/home/bazire/loreal-poc/loreal_poc/tests/performance.py:85: RuntimeWarning: Mean of empty slice
  return np.nanmean(NMEs.get(prediction_result, marks))
/home/bazire/loreal-poc/.venv/lib/python3.10/site-packages/face_alignment/api.py:147: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")


In [10]:
results

{'OpenCV': {'model': <loreal_poc.models.wrappers.OpenCVWrapper at 0x7f0ac27d8730>,
  'entire face': 0.031593074968095824,
  'face contour': 0.0524161528719839,
  'left eyebrow': nan,
  'right eyebrow': nan,
  'nose': nan,
  'left eye': nan,
  'right eye': nan,
  'mouth': nan,
  'bottom half': nan,
  'upper half': 0.04021610184657019,
  'left half': 0.662329401666445,
  'right half': 0.9675771331137769},
 'FaceAlignment': {'model': <loreal_poc.models.wrappers.FaceAlignmentWrapper at 0x7f0ac27db220>,
  'entire face': 0.06824182183674955,
  'face contour': 0.13120804359950317,
  'left eyebrow': nan,
  'right eyebrow': nan,
  'nose': nan,
  'left eye': 2.2704133968712634,
  'right eye': nan,
  'mouth': nan,
  'bottom half': 8.182222629236117e-05,
  'upper half': 0.22317472213175207,
  'left half': 0.6372027526669263,
  'right half': 0.3839864768052418}}